# Python Query Tool

## What is it

This tool lets you query information from the [Reddit Pushshift API](https://github.com/pushshift/api). With this tool you can:

- Explore how often Reddit users have posted about a given topic
- Find sample posts about a given topic
- Get top-level metrics about Reddit posts

## How can I use it

Simply add in your query into the below fields. All fields are optional. Please ensure that all fields, including blank fields, are wrapped in single quotes (''). 

Once you have added in all of your query info hit "Cell" -> "Run all" (https://screenshot.googleplex.com/mPuJGiqfKMK.png). 

## More info

For questions, comments, feature requests, more information, or a friendly chat reach out to zmpollack@.

In [ ]:
# Search by specific post IDs.
ids = ''

# Keyword search term. 
q = ''

# Exlude a keyword in the search. 
q_not = ''

# Search just post titles by keyword. 
title = ''

# Exclude a keyword in post titles. 
titlenot = ''

# Number of posts to be returned. Can be between 0 - 500. Default is 25.
size = ''

# Search by post author. 
author = ''

# Limit the search to a specfic subreddit.
subreddit = ''

# Search for submissions posted after a given date. Format must be mm/dd/yyyy.
after = ''

# Search for submissions posted before a given date. Format must be mm/dd/yyyy.
before = ''

# Limit search to posts with a given score. 
score = ''

# Limit search to posts with a given number of comments. 
num_comments = ''

In [ ]:
import requests
import json
import time
import os
import urllib

def convertTime():
    global after
    global before
    pattern = '%m/%d/%Y'
    os.environ['TZ']='UTC'
    
    if after == '':
        pass
    else:
        after = str(int(time.mktime(time.strptime(after, pattern))))
    if before == '':
        pass
    else:
        before = str(int(time.mktime(time.strptime(before, pattern))))
   
def createRequest():
    global q
    global q_not
    
    if q == '':
        pass
    else:
        q = urllib.parse.quote_plus(q)
        
    if q_not == '':
        pass
    else:
        q_not = urllib.parse.quote_plus(q_not)
    
    request = 'https://api.pushshift.io/reddit/search/submission/?'
    request += 'size=' + size + '&'
    request += "ids=" + ids + "&"
    request += "q=" + q + "&"
    request += "q:not=" + q_not + "&"
    request += "title=" + title + "&"
    request += "title:not=" + titlenot + "&"
    request += "author=" + author + "&"
    request += "subreddit=" + subreddit + "&"
    request += "after=" + after + "&"
    request += "before=" + before + "&"
    request += "score=" + score + "&"
    request += "num_comments" + num_comments + "&"
    request += 'metadata=true' 
    
    return request
    
def getResponse(request):
    r = requests.get(request)
    result = json.loads(r.content)
    return result
    
def formatResponse(args):
    
    metadata = args['metadata']
    total_results = metadata['total_results']
    results_returned = metadata['results_returned']
    
    metadata_str = """
    **Info** \n \n
    Total submissions: %s \n
    Posts returned: %s \n
    *************************** \n
    """ %(total_results, results_returned)
    
    data = args['data']

    
    i = 0 
    data_str = '**Posts** \n \n'
    
    while i < results_returned:
        author = data[i]['author']
        full_link = data[i]['full_link']
        num_comments = data[i]['num_comments']
        subreddit = data[i]['subreddit']
        title = data[i]['title']
        
        data_str += """
        Title : %s \n
        Author: %s \n
        Subreddit: %s \n
        Link: %s \n
        Number of comments: %s \n
        *************************** \n
        """ %(title, author, subreddit, full_link, num_comments)
        
        i += 1
    
    full_str = str(metadata_str) + str(data_str)
    
    return(full_str)

convertTime()

request = createRequest()

response = getResponse(request)
    
print(formatResponse(response))